# A quick notebook to check all the code imports

Though going forward, I will start writing tests, in the meanwhile here is some quick tests to see all is okay.

## Test Ising with Sampling 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
from MaxEnt import NumIsing

In [4]:
def test_pert_init(avgs = 0.05*np.ones(5), corrs = 0.002*np.triu(np.ones((5,5)),1)):  
    N = 5
    avgs = avgs # prob of every neuron firing in a window is 0.5
    corrs = corrs # prob of 2 neurons firing in the same window is 0.2 
    print("Init model")
    ising = NumIsing(N, avgs, corrs, lr=0.5,analytic=True) 
    ising.pert_init()
    print(ising.avgs, ising.corrs)
    
    pred_avgs = ising.averages(analytic=True)
    print("Predicted averages:", np.round(pred_avgs,4))
    pred_corrs = ising.correlations(analytic=True)
    print("Predicted correlations:", np.round(pred_corrs,4))

def test_MC():
    N=20
    
    avgs = 0.05*np.ones(N) # prob of every neuron firing in a window is 0.5
    corrs = 0.002*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
    print("Init model")
    ising = NumIsing(N, avgs, corrs, lr=0.5)
    ising.pert_init()
    ising.save_samples()
    print("Init averages",ising.averages(compute=True))
    
    print("State space ", 2**N)
    N_samples=1000; chains=4; N_sets=10; updates_per_set=100; M = N_samples*chains
    print(f"{N_sets} sets of {M} samples will be generated, for a total of {N_sets*M} states")
    
    print("Starting gradient ascent with sampling")
    start = time.time()
    ising.num_gradient_ascent(N_samples, chains, N_sets, updates_per_set) 
    stop = time.time()
    print("Stop grad ascent, time: ",stop-start,"s")
    
    pred_avgs = ising.averages()
    print("Predicted averages:", np.round(pred_avgs,4))
    # pred_corrs = ising.correlations()
    # print("Predicted correlations:", np.round(pred_corrs,4))


def test_correlations():
    N = 5
    avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
    corrs = 0.2*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
    
    print("Init model")
    ising = NumIsing(N, avgs, corrs, lr=0.5,analytic=True) 
    
    print("Starting grad ascent")
    ising.num_gradient_ascent() #
    print("Stop grad ascent")
    
    pred_avgs = ising.mod_avgs
    pred_corrs = ising.mod_corrs
    print("Predicted averages:", np.round(pred_avgs,4), "Predicted correlations:", np.round(pred_corrs,4),sep="\n")

    #test averages
    print("Default", ising.correlations())
    print("Analytic", ising.correlations(analytic=True))
    print("Compute",ising.correlations(compute=True))
    ising.save_samples()
    print("After more samples",ising.correlations(compute=True))

def test_samples(): #fix this
    N = 10
    print("Init model")
    ising = NumIsing(N, 0.5*np.ones(N), 0.2*np.triu(np.ones((N,N)),1), lr=0.5, analytic=True) 

    M = 500000
    chains = 10
    start = time.time()
    samples = ising.gibbs_sampling(M,chains)
    stop = time.time()
    print(f"Time to generate {chains}x{M} samples", stop-start)

    means_true = ising.averages(analytic=True)
    means_sample = np.mean(samples, axis=0)
    
    dp = 5 #decimal places
    print("True means", np.round(means_true,dp), "Sample means", np.round(means_sample,dp), sep="\n")
    print("Difference in means", means_true - means_sample, sep="\n")

    corrs_true = ising.correlations(analytic=True)
    corrs_sample = np.triu((samples.T@samples) / samples.shape[0],k=1)
    print("True correlations", np.round(corrs_true,dp), "Sample correlations", np.round(corrs_sample,dp), sep="\n")
    print("Difference in correlations", corrs_true - corrs_sample, sep="\n")

In [5]:
test_pert_init(avgs = 0.03*np.ones(5), corrs = 0.002*np.triu(np.ones((5,5)),1))

Init model
[0.03 0.03 0.03 0.03 0.03] [[0.    0.002 0.002 0.002 0.002]
 [0.    0.    0.002 0.002 0.002]
 [0.    0.    0.    0.002 0.002]
 [0.    0.    0.    0.    0.002]
 [0.    0.    0.    0.    0.   ]]
Predicted averages: [0.0355 0.0355 0.0355 0.0355 0.0355]
Predicted correlations: [[0.    0.003 0.003 0.003 0.003]
 [0.    0.    0.003 0.003 0.003]
 [0.    0.    0.    0.003 0.003]
 [0.    0.    0.    0.    0.003]
 [0.    0.    0.    0.    0.   ]]


In [6]:
test_MC()

Init model
Init averages [0.043    0.0445   0.04255  0.0435   0.035    0.050125 0.04515  0.048
 0.032    0.0425   0.0475   0.0345   0.0415   0.034    0.0435   0.053125
 0.04     0.0415   0.04795  0.040525]
State space  1048576
10 sets of 4000 samples will be generated, for a total of 40000 states
Starting gradient ascent with sampling
Stop grad ascent, time:  5.688671112060547 s
Predicted averages: [0.0329 0.0422 0.0601 0.058  0.0539 0.0504 0.0523 0.0367 0.049  0.0588
 0.0548 0.0563 0.0533 0.0689 0.0238 0.0522 0.0505 0.0371 0.0359 0.0357]


In [7]:
test_correlations()

Init model
Starting grad ascent
Stop grad ascent
Predicted averages:
[0.4826 0.4894 0.4878 0.487  0.492 ]
Predicted correlations:
[[0.     0.2097 0.2074 0.2093 0.2069]
 [0.     0.     0.2074 0.204  0.1988]
 [0.     0.     0.     0.2052 0.2038]
 [0.     0.     0.     0.     0.2064]
 [0.     0.     0.     0.     0.    ]]
Default [[0.         0.20972124 0.2073784  0.20931581 0.20692516]
 [0.         0.         0.20742646 0.2039736  0.1988193 ]
 [0.         0.         0.         0.20515899 0.20378105]
 [0.         0.         0.         0.         0.20639513]
 [0.         0.         0.         0.         0.        ]]
Analytic [[0.         0.2148272  0.2080379  0.22528282 0.2091629 ]
 [0.         0.         0.19045851 0.21458755 0.20765018]
 [0.         0.         0.         0.19483903 0.19021568]
 [0.         0.         0.         0.         0.18863419]
 [0.         0.         0.         0.         0.        ]]
Compute [[0.      0.19375 0.21125 0.2075  0.215  ]
 [0.      0.      0.198   0.1

In [27]:
test_samples()

Init model
Time to generate 10x500000 samples 3.9725329875946045
True means
[0.25369 0.17027 0.23762 0.20605 0.23671 0.30887 0.19493 0.26086 0.1286
 0.19003]
Sample means
[0.25355 0.1706  0.23853 0.20576 0.23647 0.30838 0.19477 0.26083 0.1284
 0.18993]
Difference in means
[ 1.31531687e-04 -3.27371009e-04 -9.15292932e-04  2.94867351e-04
  2.42117921e-04  4.94981806e-04  1.62135535e-04  2.87544652e-05
  2.04652259e-04  9.46266481e-05]
True correlations
[[0.      0.04147 0.04922 0.03394 0.06056 0.05539 0.04732 0.05343 0.0294
  0.03237]
 [0.      0.      0.02831 0.03373 0.03818 0.0534  0.02102 0.03671 0.02124
  0.02487]
 [0.      0.      0.      0.03862 0.04852 0.07822 0.03425 0.04646 0.03178
  0.0291 ]
 [0.      0.      0.      0.      0.03561 0.05618 0.0291  0.04784 0.02488
  0.03238]
 [0.      0.      0.      0.      0.      0.05863 0.03675 0.04773 0.01833
  0.04829]
 [0.      0.      0.      0.      0.      0.      0.05054 0.08336 0.02419
  0.04666]
 [0.      0.      0.      0.      0.

## Test Ising without sampling

In [8]:
from MaxEnt import Ising

In [9]:
def fit_example():
    N = 5
    avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
    corrs = 0.2*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
    
    print("Init model")
    ising = Ising(N, avgs, corrs, lr=0.5) 
    
    print("Starting grad ascent")
    start = time.time()
    for _ in range(10):
        ising.gradient_ascent() # 500 steps 
    print(f"Stop grad ascent: {time.time()-start}s")
    
    pred_avgs = ising.averages()
    pred_corrs = ising.correlations()
    print("Predicted averages:", pred_avgs, "Predicted correlations:", pred_corrs,sep="\n")
    print(f"P({ising.states[0]})={ising.p(ising.states[0])}")

def av_time_grad_ascent():
    # Calculate average times
    reps = 50
    startN = 8
    stopN = 10 #inclusive
    Ns = np.arange(startN,stopN+1)
    times = np.zeros( (reps,len(Ns)) )
    for i in range(reps):
        if not (i+1)%10:
            print("Repetitions: ", i+1)
        for N in Ns:
            avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
            corrs = 0.2*np.triu(np.ones((N,N)),1) # prob of 2 neurons firing in the same window is 0.2 
            ising = Ising(N, avgs, corrs, lr=0.5) 
            start = time.time()
            ising.gradient_ascent() # 500 steps 
            stop = time.time()
            times[i,N-startN]=stop-start
    
    av_times = np.mean(times,0)
    std_times = np.std(times,0)

    plt.plot(Ns, av_times, "k.")
    plt.plot(Ns, av_times+2*std_times/np.sqrt(reps), "r_")
    plt.plot(Ns, av_times-2*std_times/np.sqrt(reps), "r_")
    plt.title("Time for 100 steps of grad. ascent vs. system size")
    plt.xlabel("System size")
    plt.ylabel("Time (seconds)")
    plt.show()

In [10]:
fit_example()

Init model
Starting grad ascent
Stop grad ascent: 1.081038236618042s
Predicted averages:
[0.49919728 0.49919728 0.49919728 0.49919728 0.49919728]
Predicted correlations:
[[0.         0.20039872 0.20039872 0.20039872 0.20039872]
 [0.         0.         0.20039872 0.20039872 0.20039872]
 [0.         0.         0.         0.20039872 0.20039872]
 [0.         0.         0.         0.         0.20039872]
 [0.         0.         0.         0.         0.        ]]
P([0 0 0 0 0])=7.359000155837554e-07


In [11]:
av_time_grad_ascent()

Repetitions:  10
Repetitions:  20
Repetitions:  30
Repetitions:  40
Repetitions:  50


KeyboardInterrupt: 

## Test K-Pairwise without sampling

In [1]:
from MaxEnt import KPair 

In [ ]:
N = 7
avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
corrs = np.triu(np.ones((N,N)),1)/3 # prob of 2 neurons firing in the same window is 0.333
p_k = np.ones(N+1)/(N+1) # prob of k neurons firing is uniform

print("Init model")
kpair = KPair(N, avgs, corrs, p_k, lr=0.5) 

In [49]:
print("Starting grad ascent")
start = time.time()
for _ in range(4):
    kpair.gradient_ascent() # 500 steps 
print(f"Stop grad ascent: {time.time()-start}s")

Starting grad ascent
Stop grad ascent: 1.0277791023254395s


In [50]:
pred_avgs = kpair.averages()
pred_corrs = kpair.correlations()
pred_p_k = kpair.prob_k()
print("Predicted:\naverages:", pred_avgs, "correlations:", pred_corrs, "p(K)", pred_p_k,sep="\n")
print(f"P({kpair.states[0]})={kpair.p(kpair.states[0])}")

Predicted:
averages:
[0.5 0.5 0.5 0.5 0.5 0.5 0.5]
correlations:
[[0.         0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333]
 [0.         0.         0.33333333 0.33333333 0.33333333 0.33333333
  0.33333333]
 [0.         0.         0.         0.33333333 0.33333333 0.33333333
  0.33333333]
 [0.         0.         0.         0.         0.33333333 0.33333333
  0.33333333]
 [0.         0.         0.         0.         0.         0.33333333
  0.33333333]
 [0.         0.         0.         0.         0.         0.
  0.33333333]
 [0.         0.         0.         0.         0.         0.
  0.        ]]
p(K)
[0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
P([0 0 0 0 0 0 0])=0.12499999999999999


## Test threewise dist

In [12]:
from MaxEnt import ThreeWise 

In [13]:
N = 4
h = np.random.random_sample((N))
J = np.triu( np.random.random_sample((N,N)), 1)
K = np.zeros((N,N,N))

alpha = 1
for i in range(N-2):
    for j in range(i+1,N-1):
        for k in range(j+1, N):
            K[i,j,k] = alpha 
            alpha /= 10
ex = ThreeWise(N, h, J, K)

#calc 3 wise correlations
for i in range(N-2):
    for j in range(i+1,N-1):
        for k in range(j+1, N):
            print(f"Correlation between {i},{j},{k}\n", ex.expectation(lambda s: s[:,i]*s[:,j]*s[:,k]) )

Correlation between 0,1,2
 0.0016569727567211031
Correlation between 0,1,3
 0.010221243702375963
Correlation between 0,2,3
 0.006279566482332794
Correlation between 1,2,3
 0.0032954507812045473


## Test coarse models

In [14]:
from MaxEnt import Independent, PopCount

In [15]:
N = 10
avgs = 0.5*np.ones(N) # prob of every neuron firing in a window is 0.5
print("Init model")
start = time.time()
ind = Independent(N, avgs) 
print(f"Time: {time.time()-start}s")

Init model
Time: 0.024595975875854492s


In [16]:
pred_avgs = ind.averages()
print("Predicted averages:", pred_avgs, sep="\n")
print(f"P({ind.states[0]})={ind.p(ind.states[0])}")

Predicted averages:
[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
P([0 0 0 0 0 0 0 0 0 0])=0.0009765625


In [17]:
samples = ind.get_samples()

In [18]:
np.mean(samples,axis=0)

array([0.503675 , 0.4933225, 0.501925 , 0.49805  , 0.50539  , 0.494725 ,
       0.50206  , 0.5007925, 0.499155 , 0.5023725])

In [19]:
# Pop Count
N = 10
p_K = np.ones(N+1)/(N+1) 
print("Init model")
start = time.time()
pop = PopCount(N,p_K)
print(f"Time: {time.time()-start}s")

Init model
Time: 0.02677297592163086s


In [20]:
pred_p_k = pop.prob_k()
print("P(K):", pred_p_k, sep="\n")
print(f"P({pop.states[0]})={pop.p(pop.states[0])}")

P(K):
[0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
P([0 0 0 0 0 0 0 0 0 0])=0.09090909090909091


In [21]:
pop.expectation(lambda s: s[:,0]*s[:,1])

0.33333333333333326

In [22]:
samples = pop.get_samples()

In [23]:
sample_p_K = np.bincount(np.sum(samples,axis=1).astype(int))/len(samples)
print(p_K,sample_p_K,sep="\n")

[0.09090909 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909
 0.09090909 0.09090909 0.09090909 0.09090909 0.09090909]
[0.0931075 0.0928925 0.0919475 0.09159   0.0900375 0.0896675 0.0884875
 0.0890925 0.088515  0.089755  0.0949075]


In [24]:
from numba import njit

In [25]:
class test:
    
    @staticmethod
    def call_other():
        test.call_this()
        
    @staticmethod
    def call_this():
        print("hello_world")
    

In [26]:
test.call_other()

hello_world
